## This project involves creating a daily report that is crucial for both salespeople within the company and partner clients. The report contains detailed information about each order, including the order number, status, scheduling dates for collection and delivery, lead time, cancellations, returns, delivery dates, and more. It aims to provide a comprehensive overview of the order lifecycle and key details relevant to sales and customer service operations.

# 0.0 IMPORTS

In [1]:
import pandas as pd
import datetime
import pyautogui
import glob
import win32com.client
from datetime import datetime,date
pyautogui.FAILSAFE = True

In [2]:
#connection with SAP

SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)
now = datetime.now()
data_atual = date.today()

# 2.0 EXTRACTING DATABASES

In [3]:
# Extraction 002N (EXPECTED DATE ACCORDING TO LEAD TIME) via SAP SCRIPT


session.findById("wnd[0]").maximize()
session.findById("wnd[0]/tbar[0]/okcd").text = "002n"
session.findById("wnd[0]").sendVKey (0)
session.findById("wnd[0]/tbar[1]/btn[17]").press()
session.findById("wnd[1]/usr/txtV-LOW").text = "XXXXXX"
session.findById("wnd[1]/usr/txtENAME-LOW").text = ""
session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 12
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "Atualização_PBI/CS_Extracao_AcompanhamentoDiario"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "dt_prevista.xlsx"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 16
session.findById("wnd[1]/tbar[0]/btn[11]").press()
session.findById("wnd[0]/tbar[0]/btn[3]").press()
session.findById("wnd[0]/tbar[0]/btn[3]").press()



In [15]:
# Extraction VA05 (Order block Information) via SAP SCRIPT


session.findById("wnd[0]").maximize()
session.findById("wnd[0]/tbar[0]/okcd").text = "va05"
session.findById("wnd[0]").sendVKey (0)
session.findById("wnd[0]/tbar[1]/btn[17]").press()
session.findById("wnd[1]/usr/txtV-LOW").text = "XXXXXXXXX"
session.findById("wnd[1]/usr/txtENAME-LOW").text = ""
session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 13
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[8]").press()
session.findById("wnd[0]/tbar[1]/btn[33]").press()
session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").currentCellRow = 18
session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = "18"
session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").clickCurrentCell()
session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[1]").select()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "Atualização_PBI/CS_Extracao_AcompanhamentoDiario"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "va05.xlsx"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 9
session.findById("wnd[1]/tbar[0]/btn[11]").press()
session.findById("wnd[0]/tbar[0]/btn[3]").press()
session.findById("wnd[0]/tbar[0]/btn[3]").press()


# 2.0 LOADING THE DATABASES

In [16]:
# Load Excel files into separate DataFrames
zva05 = pd.read_excel('zva05.xlsx')
zbrsd038  = pd.read_excel('038.xlsx')
zbrle002n = pd.read_excel('dt_prevista.xlsx')
va05 = pd.read_excel('va05.xlsx')
baseSKU = pd.read_excel('Base de SKU.xlsx')
baseClientes = pd.read_excel('Base de Clientes.xlsx')
zbrsd027 = pd.read_excel('027.xlsx')

# Lista de arquivos Excel na pasta
pasta = 'Documentos/Planilhas de Agendamento'
arquivos_excel = glob.glob(pasta + '/*.xlsx')

# Initialize empty list for DataFrames
df_list = []


# Read Excel files from folder and append DataFrames to df_list
sheet_name = 'Agendamento '
for arquivo in arquivos_excel:
    df = pd.read_excel(arquivo, sheet_name=sheet_name,skiprows=4, header=0)
    df_list.append(df)
    
# Concatenate DataFrames into a single DataFrame
df_ag = pd.concat(df_list, ignore_index=True)

C:\ProgramData\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\ProgramData\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\ProgramData\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\ProgramData\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\ProgramData\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


# 3.0 DATA PREPARATION

In [17]:
# Removing blank rows
zva05['Doc.venda'] = zva05['Doc.venda'].fillna(0)
zbrsd038['Documento de vendas'] = zbrsd038['Documento de vendas'].fillna(0)
df_ag['Ordem Venda'] = df_ag['Ordem Venda'].fillna(0)
zbrsd027['Ordem Venda'] = zbrsd027['Ordem Venda'].fillna(0)

In [18]:
# Transformation to INT to Create Key
zva05['Doc.venda'] = zva05['Doc.venda'].astype(int)
zbrsd038['Documento de vendas'] = zbrsd038['Documento de vendas'].astype(int)
df_ag['Ordem Venda'] = df_ag['Ordem Venda'].astype(int)
va05['Documento de vendas'] = va05['Documento de vendas'].astype(int)
zbrle002n['Número de nota fiscal eletrônica'] = zbrle002n['Número de nota fiscal eletrônica'].astype(int)
zbrsd027['Ordem Venda'] = zbrsd027['Ordem Venda'].astype(int)

# Renaming columns for merging
zva05 = zva05.rename(columns={'Doc.venda': 'Ordem de Venda', 'Remessa': 'Delivery', 'Referência cliente': 'Nº pedido cliente', 'Sold-to Code': "Cód Cliente", 'Sold-to Name': "Cliente",'Material':'SKU', 'Nº do material':'Descrição SKU', 'Conf Qty': 'Volume (Cxs)', 'Return':'Devolução', 'Dt.carreg.': 'Data Agendada Coleta', 'Data doc.': 'Data Criação Pedido', 'Cen.': 'Centro', 'Del Date':'Data Criação Delivery' })
zbrsd038 = zbrsd038.rename(columns={'Documento de vendas': 'Ordem de Venda', 'Número da remessa' : 'Delivery', 'Número de nota fiscal eletrônica': 'NF'})
df_ag = df_ag.rename(columns={'Ordem Venda': 'Ordem de Venda', 'Remessa': 'Delivery'})
va05 = va05.rename(columns={'Documento de vendas': 'Ordem de Venda', 'Bloqueio remessa': 'Bloqueio EDI'})
zbrle002n = zbrle002n.rename(columns={'Número de nota fiscal eletrônica': 'NF'})
baseSKU = baseSKU.rename(columns={'De/SKU': 'SKU'})
baseClientes = baseClientes.rename(columns={'Cliente': 'Cód Cliente'})
zbrsd027 = zbrsd027.rename(columns={'Ordem Venda': 'Ordem de Venda'})

# Deleting unnecessary columns
colunas_para_excluir = ['Ship-to Code', 'Ship-to Name', 'SaídaMerc', 'EscrV', 'Sales Group']
zva05 = zva05.drop(columns=colunas_para_excluir)

# 4.0 MERGE BETWEEN THE TABLES

## 4.1 MERGE (1)  zva05 + zbrsd038 

In [19]:

# Agrupando a zva05 pela ordem
zbrsd038_grouped = zbrsd038.groupby('Delivery').agg({'NF':'first','Data de carregamento':'first', 'Data Entrega Informação':'first'}).reset_index()

# Merge zva05 + zbrsd038
zva05_zbrsd038 = zva05.merge(zbrsd038_grouped, on='Delivery',how= 'left')


## 4.2 MERGE (2)  zva05 + zbrsd038 + zbrle002n

In [20]:
# Agrupando a 002 pela NF
zbrle002n_grouped = zbrle002n.groupby('NF').agg({'Data Prevista':'first'}).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n
zva05_zbrsd038_zbrle002n = zva05_zbrsd038.merge(zbrle002n_grouped, on='NF',how= 'left')

## 4.3 MERGE (3)  zva05 + zbrsd038 + zbrle002n +  Agendamento

In [21]:
# Agrupando a Agenda pela Ordem de Venda
df_ag_grouped = df_ag.groupby('Ordem de Venda').agg({'Data Agenda':'first','Status da Agenda':'first', 'Observação Nfe':'first'}).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n + Agendamentos
zva05_zbrsd038_zbrle002n_dfag = zva05_zbrsd038_zbrle002n.merge(df_ag_grouped, on='Ordem de Venda',how= 'left')

## 4.4 MERGE (4)  zva05 + zbrsd038 + zbrle002n + Agendamento + va05

In [22]:
# Agrupando a VA05 pela Ordem de Venda
va05_grouped = va05.groupby('Ordem de Venda').agg({'Bloqueio EDI':'first'}).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n + Agendamentos + VA05
zva05_zbrsd038_zbrle002n_dfag_va05 = zva05_zbrsd038_zbrle002n_dfag.merge(va05_grouped, on='Ordem de Venda',how= 'left')

## 4.5 MERGE (5)  zva05 + zbrsd038 + zbrle002n + va05 + Agendamento + va05 + baseSKU

In [23]:
# Agrupando a base de SKU pelo numero do SKU
SKU_grouped = baseSKU.groupby('SKU').agg({'Sub Marca':'first'}).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n + Agendamentos + VA05 + base SKU
zva05_zbrsd038_zbrle002n_dfag_va05_SKU = zva05_zbrsd038_zbrle002n_dfag_va05.merge(SKU_grouped, on='SKU',how= 'left')

###### 4.6 MERGE (6)  zva05 + zbrsd038 + zbrle002n + va05 + Agendamento + va05 + baseSKU + baseClientes

In [24]:
# Agrupando a base de clientes pelo código do cliente
Clientes_grouped = baseClientes.groupby('Cód Cliente').agg({'Cidade':'first','U.F.':'first', 'Regional':'first', 'Vendedor':'first', 'Segmentação':'first' }).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n + Agendamentos + VA05 + base SKU + base de clientes
zva05_zbrsd038_zbrle002n_dfag_va05_SKU_Clientes = zva05_zbrsd038_zbrle002n_dfag_va05_SKU.merge(Clientes_grouped, on='Cód Cliente',how= 'left')

## 4.7 MERGE (7)  zva05 + zbrsd038 + zbrle002n + va05 + Agendamento + va05 + baseSKU + baseClientes + zbrsd027

In [25]:
# Agrupando a base zbrsd027 pela ordem de venda
zbrsd027_grouped = zbrsd027.groupby('Ordem de Venda').agg({'Status Pedido':'first'}).reset_index()

# Merge zva05 + zbrsd038 + zbrle002n + Agendamentos + VA05 + base SKU + base de clientes + 027
zva05_zbrsd038_zbrle002n_dfag_va05_SKU_Clientes_027 = zva05_zbrsd038_zbrle002n_dfag_va05_SKU_Clientes.merge(zbrsd027_grouped, on='Ordem de Venda',how= 'left')

# 5.0 EXPORT FINAL FILE BASES

In [26]:
zva05_zbrsd038_zbrle002n_dfag_va05_SKU_Clientes_027.to_excel('Extracao_Report_Diario_2024.xlsx', index=False)